<a href="https://colab.research.google.com/github/liuxx479/stickgame/blob/main/stick_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy import *
import numpy as np
import copy
import hashlib
from collections import defaultdict

In [2]:
def hash_list(lst):
    lst_bytes = str(list(lst)).encode()  # Convert to string and encode
    return hashlib.sha256(lst_bytes).hexdigest()  # Generate SHA-256 hash

In [3]:
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

In [4]:
def init_board (lv):
    '''initialize the board'''
    out=np.arange(lv)+1
    return out

print ('test init_board(4):', init_board(4))

test init_board(4): [1 2 3 4]


In [5]:
def split_num (num): ## e.g. 5
    '''split each row to possible new outcomes'''
    # out=[]
    out=[[],] ## can always entirely cross the whole row, so start with a []
    if num==1:
        return out
    for inum in np.arange(num):  ## initial idx
        n0=inum
        for jnum in np.arange(inum+1,num): ## final idx
            n1=num-jnum
            if n1<n0: ## [2,1] and [1,2] are the same, stop computing
                break
            if n0!=0:
                out.append([n0,n1])
            else:
                out.append([n1])
    return out

print ('test split 5:', split_num(5))

test split 5: [[], [4], [3], [2], [1], [1, 3], [1, 2], [1, 1], [2, 2]]


In [9]:
def init_win_board():
    winning_board_cache = {}
    for iboard in [[1,]]:#,[2,2]]:#[1,2,3]]:
        winning_board_cache[hash_list(iboard)] = list(iboard)
    return winning_board_cache

winning_board_cache0=init_win_board()
print (winning_board_cache0.values())

dict_values([[1]])


In [11]:
def clean_board (board):
    '''remve double 1, and sort numbers'''
#     print ('sum(board==1)',sum(board==1))
    board=np.array(board)

    odd1s=sum(board==1)%2 ## True if odd number of 1s
#     print (odd1s)
    board=board[board!=1]
    if odd1s:
        board=np.append(board,[1])
    ## second, sort the array, so only split the ones that are unique
    board=np.sort(board)
    return list(board)

list_unique = lambda out: [list(x) for x in set(tuple(x) for x in out)]

def possible_out (board, winning_board_cache=winning_board_cache0):
    '''produce all possible outcomes (after 1 player) for a current board'''
    board=clean_board(board)
    num_unique, idx_unique=np.unique(board, return_index=1)
    out=[]
    for inum, iidx in zip(num_unique, idx_unique):
        splits=split_num (inum)
        for isplit in splits:
            iboard=list(copy.deepcopy(board))
            iboard=np.delete(iboard,iidx)
            iboard=np.concatenate([iboard,isplit]).astype(int)
            iboard=clean_board(iboard)

            if hash_list(iboard) in winning_board_cache0:
                return list_unique([iboard]) ## if there's a board that can force others to lose, then stop here

            out.append(iboard)
    out =  list_unique(out) ## only count unique possibilities e.g. [1,2] and [1,2,1,1] would be the same after clean up
    return out

def status(board, winning_board_cache=winning_board_cache0):
    # '''return True when reaching the leaf, or board clears []'''
    '''return True when winning, i.e. there is [1] or [2,2] left; False otherwise (unclear)'''
    if len(board)==0 or hash_list(board) in winning_board_cache:
        return True
    else:
        return False

## test code
board0=init_board(3)
print ('Test initial board',board0)
boards=possible_out (board0)
print ('Test possible child boards',boards)
# for iboard in boards:
#     print (iboard, status(iboard))
print ('Test status of the board (True=the one just finished win, or the one about to move lose):')
for itest in [[1,],[2,2],[3]]:
    print (itest, status(itest))

Test initial board [1 2 3]
Test possible child boards [[1, 3], [1, 2], [2], [2, 3], [1, 2, 2], [3]]
Test status of the board (True=the one just finished win, or the one about to move lose):
[1] True
[2, 2] False
[3] False


In [46]:
# def build_stick_tree(board):
#     """Builds a stick game tree where each node splits into all possible new boads"""
#     root = TreeNode(board)

#     def recursive_add(node):
#         if status(node.value):
#             return
#         boards=possible_out (node.value)
#         for iboard in boards:
#             if len(iboard)==0:
#                 break
#             ichild=TreeNode(iboard)
#             node.add_child(ichild)
#             recursive_add(ichild)

#     recursive_add(root)
#     return root


def build_stick_tree_cache (board, node_cache=None, winning_board_cache=None):
    """Builds a stick game tree where each node splits into all possible new boads"""

    if node_cache is None:
        node_cache = {}
    if winning_board_cache is None:
        winning_board_cache = {}

    hash_board=hash_list(board)

    if hash_board in node_cache:
        return node_cache[hash_board]

    root = TreeNode(board)
    node_cache[hash_board] = root

    def recursive_add(node):
        if status(node.value,winning_board_cache=winning_board_cache):# and len(node.value)==0:
            return
        boards=possible_out (node.value,winning_board_cache=winning_board_cache)
        for iboard in boards:
            if len(iboard)==0:
                break
            ihash_board=hash_list(iboard)
            if ihash_board in node_cache:
                ichild=node_cache[ihash_board]
            else:
                ichild=TreeNode(iboard)
                recursive_add(ichild)
                node_cache[ihash_board]=ichild

            node.add_child(ichild)

    recursive_add(root)
    return root

def print_tree(node, depth=0):
    """Prints the tree structure."""
    print("  " * depth + str(node.value))
    for child in node.children:
        print_tree(child, depth + 1)


# Example usage
lv=3  # Change this number to test different values
board=init_board(lv)

## cache tree
node_cache = {}
root_cache = build_stick_tree_cache (board, node_cache)
print("Stick Tree (Cache):")
print_tree(root_cache)

## no cache tree
# root = build_stick_tree(board)
# print("Stick Tree:")
# print_tree(root)

Stick Tree (Cache):
[1 2 3]
  [1, 3]
    [1]
  [1, 2]
    [1]
  [2]
    [1]
  [2, 3]
    [1, 2]
      [1]
    [2]
      [1]
    [3]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
    [1, 3]
      [1]
  [1, 2, 2]
    [1, 2]
      [1]
    [2]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
  [3]
    [1]


In [102]:
# %timeit build_stick_tree(board)
# %timeit build_stick_tree_cache (board)

In [47]:
### now walk through the tree to find winning path
def find_leaf_levels_and_paths(node, level=0, path="", leaves=None):
    """Finds all leaf nodes, their levels, and paths in the tree."""
    if leaves is None:
        leaves = []

    if not node.children:  # If it's a leaf node
        leaves.append((node.value, level, path))

    for child in node.children:
        find_leaf_levels_and_paths(child, level + 1, path + f"->{child.value}", leaves)

    return leaves

# Example usage:
leaves = find_leaf_levels_and_paths(root_cache)
print("Possible paths for stick game level", board)
for value, lvl, path in leaves:
    print(f"Leaf depth: {lvl} ({['LOST', 'WIN '][lvl%2]}), Path: {path}")

Possible paths for stick game level [1 2 3]
Leaf depth: 2 (LOST), Path: ->[1, 3]->[1]
Leaf depth: 2 (LOST), Path: ->[1, 2]->[1]
Leaf depth: 2 (LOST), Path: ->[2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[1, 2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[3]->[1]
Leaf depth: 4 (LOST), Path: ->[2, 3]->[2, 2]->[1, 2]->[1]
Leaf depth: 4 (LOST), Path: ->[2, 3]->[2, 2]->[2]->[1]
Leaf depth: 3 (WIN ), Path: ->[2, 3]->[1, 3]->[1]
Leaf depth: 3 (WIN ), Path: ->[1, 2, 2]->[1, 2]->[1]
Leaf depth: 3 (WIN ), Path: ->[1, 2, 2]->[2]->[1]
Leaf depth: 4 (LOST), Path: ->[1, 2, 2]->[2, 2]->[1, 2]->[1]
Leaf depth: 4 (LOST), Path: ->[1, 2, 2]->[2, 2]->[2]->[1]
Leaf depth: 2 (LOST), Path: ->[3]->[1]


In [48]:
### after building a tree, search all nodes, if each the node has only even number in leaf depth, then it's winning board, store in cache

def find_even_leaf_nodes(root, cache=None):
    """Searches the tree and stores nodes whose leaf depths are all even."""
    if cache is None:
        cache = {}

    def check_and_store(node, parent_level=0):
        """Recursively checks if (leaf depth - child depth) is all even."""
        if not node.children:  # If it's a leaf node
            return [parent_level + 1]  # Leaf depth relative to tree

        leaf_levels = []
        for child in node.children:
            child_level = parent_level + 1  # Child level relative to tree
            child_leaf_levels = check_and_store(child, child_level)

            # Adjust leaf levels relative to the child
            adjusted_leaf_levels = [leaf_level - child_level for leaf_level in child_leaf_levels]
            leaf_levels.extend(adjusted_leaf_levels)

        # If all (leaf level - child level) values are even, store and stop deeper recursion
        if all(level % 2 == 0 for level in leaf_levels):
            # cache.add(node.value)
            cache[hash_list(node.value)]=list(node.value)
            return []  # Stop deeper exploration

        return leaf_levels  # Continue deeper

    check_and_store(root)
    return cache

# Example usage:
winning_board_cache=init_win_board()
print ("Winning board values (before find_even_leaf_nodes):\n", winning_board_cache.values())

lv=3
board=init_board(lv)

## cache tree
root_cache = build_stick_tree_cache (board, winning_board_cache=winning_board_cache)

print("\nStick Tree (Cache):")
print_tree(root_cache)

even_leaf_cache = find_even_leaf_nodes(root_cache, cache=winning_board_cache)
print ("Winning board values (after find_even_leaf_nodes):\n", winning_board_cache.values())
print("even_leaf_cache:", even_leaf_cache.values())

Winning board values (before find_even_leaf_nodes):
 dict_values([[1]])

Stick Tree (Cache):
[1 2 3]
  [1, 3]
    [1]
  [1, 2]
    [1]
  [2]
    [1]
  [2, 3]
    [1, 2]
      [1]
    [2]
      [1]
    [3]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
    [1, 3]
      [1]
  [1, 2, 2]
    [1, 2]
      [1]
    [2]
      [1]
    [2, 2]
      [1, 2]
        [1]
      [2]
        [1]
  [3]
    [1]
Winning board values (after find_even_leaf_nodes):
 dict_values([[1], [2, 2], [1, 2, 3]])
even_leaf_cache: dict_values([[1], [2, 2], [1, 2, 3]])


In [49]:
### map out lv N tree, then update even_leaf_cache, then back to lv N-1 see if it's in winning boards, if not, then losing

winning_board_cache = init_win_board()
node_cache = {}

def play_game (lv, node_cache={}, winning_board_cache=winning_board_cache):
    board=init_board(lv)
    root_cache = build_stick_tree_cache (board, node_cache, winning_board_cache)
    even_leaf_cache = find_even_leaf_nodes(root_cache, cache=winning_board_cache)
    return root_cache


for lv in [1,2,3,4,5]:
    iroot=play_game(lv, node_cache={}, winning_board_cache=winning_board_cache)
    # leaves = find_leaf_levels_and_paths(iroot)
    # print("Possible paths for stick game level", lv)
    # for value, lvl, path in leaves:
    #     print(f"Leaf depth: {lvl} ({['LOST', 'WIN '][lvl%2]}), Path: {path}")

    print("Winning boards (lv %s)"%(lv), winning_board_cache.values())
    # print ("\n")

# print ('using full cache')
# for lv in [2,3, 4]:
#     iroot=play_game(lv, winning_board_cache=winning_board_cache)
#     leaves = find_leaf_levels_and_paths(iroot)
#     print("Possible paths for stick game level", lv)
#     for value, lvl, path in leaves:
#         print(f"Leaf depth: {lvl} ({['LOST', 'WIN '][lvl%2]}), Path: {path}")
#     print("Winning boards (lv %s)"%(lv), winning_board_cache.values())
#     print ("\n")
#     even_leaf_cache = find_even_leaf_nodes(iroot, cache=winning_board_cache)

Winning boards (lv 1) dict_values([[1]])
Winning boards (lv 2) dict_values([[1]])
Winning boards (lv 3) dict_values([[1], [2, 2], [1, 2, 3]])
Winning boards (lv 4) dict_values([[1], [2, 2], [1, 2, 3], [3, 3]])
Winning boards (lv 5) dict_values([[1], [2, 2], [1, 2, 3], [3, 3]])


In [50]:
def check_winning (board, winning_board_cache=winning_board_cache):
    '''if the initial board is in winning board, then 2nd player win, first player lost'''
    if hash_list(board) not in winning_board_cache:
        return True
    else:
        return False

for lv in [1,2,3,4,5]:
    first_win=check_winning(init_board(lv))
    print ('Lv %i board, first player %s'%(lv, ['LOSES','WINS'][first_win]))
### level 6 already hard to compute... maybe need to put losing board as well
### or walk the tree better/more efficiently

Lv 1 board, first player LOSES
Lv 2 board, first player WINS
Lv 3 board, first player LOSES
Lv 4 board, first player WINS
Lv 5 board, first player WINS


Need to optimize more.. and I think lv 5 should lose, but unsure how to walk it better, spent too much time on this already. Stop here for now.